In [23]:
import psycopg2
from pprint import pprint



def create_db(cur):
    cur.execute ("""
    CREATE TABLE IF NOT EXISTS clients(
    id SERIAL PRIMARY KEY,
    name VARCHAR(60) NOT NULL,
    surname VARCHAR(60) NOT NULL,
    email VARCHAR(60) UNIQUE NOT NULL
    );
    """)


    cur.execute("""
    CREATE TABLE IF NOT EXISTS telephone(
    id SERIAL PRIMARY KEY,
    clients_id INTEGER NOT NULL REFERENCES clients(id),
    client_telephone VARCHAR(30) UNIQUE 
    );
    """)


def add_client(cur, name, surname, email):
    cur.execute("""
    INSERT INTO clients(name, surname, email) VALUES(%s, %s, %s);
    """, (name, surname, email))


def add_phone(cur, clients_id, client_telephone):
    cur.execute("""
    INSERT INTO telephone(clients_id, client_telephone) VALUES(%s, %s);
    """, (clients_id, client_telephone))


def change_client():
    
    print("Для изменения информации о клиенте, пожалуйста, введите нужную Вам команду.\n "
        "1 - изменить имя; 2 - изменить фамилию; 3 - изменить e-mail; 4 - изменить номер телефона; 5 - удалить номер телефона клиента; 6 - удалить клиента из базы данных; 7 - выйти, если не требуется изменений")    
    while True:

        command = input()

        # Изменяем информацию о клиенте

        if command == '1':

            id_changing_name = input("Введите id клиента имя которого хотите изменить: ")
            name_changing = input("Введите имя для изменения: ")
            cur.execute("""
            UPDATE clients SET name=%s WHERE id=%s;
            """, (name_changing, id_changing_name))
            break
        elif command == '2':

            id_changing_surname = input("Введите id клиента фамилию которого хотите изменить: ")
            surname_changing = input("Введите фамилию для изменения: ")
            cur.execute("""
            UPDATE clients SET surname=%s WHERE id=%s;
            """, (surname_changing, id_changing_surname))
            break
        elif command == '3':

            id_changing_email = input("Введите id клиента e-mail которого хотите изменить: ")
            email_changing = input("Введите e-mail для изменения: ")
            cur.execute("""
            UPDATE clients SET email=%s WHERE id=%s;
            """, (email_changing, id_changing_email))
            break
        elif command == '4':

            old_phonenumber = input("Введите номер телефона который Вы хотите изменить: ")
            phonenumber_changing = input("Введите новый номер телефона, который заменит собой старый: ")
            cur.execute("""
            UPDATE telephone SET client_telephone = %s WHERE client_telephone = %s;
            """, (phonenumber_changing, old_phonenumber))
            break

        # Удаление номера телефона существующего клиента

        elif command == '5':

            id_deleting_telephone = input("Введите id клиента номер телефона которого хотите удалить: ")
            telephone_deleting = input("Введите номер телефона который хотите удалить: ")
            cur.execute("""
            DELETE FROM telephone WHERE clients_id = %s AND client_telephone = %s
            """, (id_deleting_telephone, telephone_deleting))
            break

        #Удаление клиента из базы данных

        elif command == '6':

            id_deleting_client = input("Введите id клиента которого хотите удалить: ")
            surname_deleting = input("Введите фамилию клиента которого хотите удалить: ")
            cur.execute("""
            DELETE FROM telephone WHERE clients_id = %s
            """, (id_deleting_client))

            cur.execute("""
            DELETE FROM clients WHERE id = %s AND surname=%s
            """, (id_deleting_client, surname_deleting))
            break

        elif command =='7':
            return
        
        print("К сожалению, Вы ввели неправильную команду, пожалуйста, повторите ввод")


# Поиск клиента по его данным


def find_client():
    print("Для поиска информации о клиенте, пожалуйста, введите команду, где:\n "
          "1 - найти по имени; 2 - найти по фамилии; 3 - найти по e-mail; 4 - найти по номеру телефона; 8 - прервать поиск")
    while True:
        command_finding = input("Введите команду для поиска информации о клиенте: ")

        if command_finding == '1':
            name_finding = input("Введите имя для поиска информации о клиенте: ")
            cur.execute("""
            SELECT cl.id, name, surname, email, client_telephone
            FROM clients AS cl
            JOIN telephone AS t ON t.clients_id = cl.id
            WHERE name=%s
            """, (name_finding,))
            print(cur.fetchall())
            break

        elif command_finding == '2':
            surname_finding = input("Введите фамилию для поиска информации о клиенте: ")
            cur.execute("""
            SELECT cl.id, name, surname, email, client_telephone
            FROM clients AS cl
            LEFT JOIN telephone AS t ON t.clients_id = cl.id
            WHERE surname=%s
            """, (surname_finding,))
            print(cur.fetchall())
            break

        elif command_finding == '3':
            email_finding = input("Введите email для поиска информации о клиенте: ")
            cur.execute("""
            SELECT cl.id, name, surname, email, client_telephone
            FROM clients AS cl
            LEFT JOIN telephone AS t ON t.clients_id = cl.id
            WHERE email=%s
            """, (email_finding,))
            print(cur.fetchall())
            break

        elif command_finding == '4':
            telephone_finding = input("Введите номер телефона для поиска информации о клиенте: ")
            cur.execute("""
            SELECT cl.id, name, surname, email, client_telephone
            FROM clients AS cl
            LEFT JOIN telephone AS t ON t.clients_id = cl.id
            WHERE client_telephone=%s
            """, (telephone_finding,))
            print(cur.fetchall())
            break

        elif command_finding =='8':
            return
        print("К сожалению, Вы ввели неправильную команду, пожалуйста, повторите ввод")


# Проверка

def check():
    cur.execute("""
    SELECT * FROM clients;
    """)
    pprint(cur.fetchall())

    cur.execute("""
    SELECT * FROM telephone;
    """)
    pprint(cur.fetchall())


conn = psycopg2.connect(host ="localhost", database = "HOMEWORK", user = "postgres", password = "Bilyk.elena", port = "5432")
with conn.cursor() as cur:



    create_db(cur)
    add_client(cur, 'Иван', 'Путров', 'pe@m.ru')
    add_client(cur, 'Мария', 'Сидорова', 'si@m.ru')
    add_client(cur, 'Генадий', 'Фролов', 'fr@v.ru')

    add_phone(cur, 1, '777777')
    add_phone(cur, 2, '222222')
    add_phone(cur, 3, '333333')
    add_phone(cur, 1, '666666')

    change_client()

    find_client()

    print()

    check()

    
conn.commit()

conn.close()


Для изменения информации о клиенте, пожалуйста, введите нужную Вам команду.
 1 - изменить имя; 2 - изменить фамилию; 3 - изменить e-mail; 4 - изменить номер телефона; 5 - удалить номер телефона клиента; 6 - удалить клиента из базы данных; 7 - выйти, если не требуется изменений
Для поиска информации о клиенте, пожалуйста, введите команду, где:
 1 - найти по имени; 2 - найти по фамилии; 3 - найти по e-mail; 4 - найти по номеру телефона; 8 - прервать поиск
[(1, 'Иван', 'Путров', 'pe@m.ru', '777777'), (1, 'Иван', 'Путров', 'pe@m.ru', '666666')]

[(1, 'Иван', 'Путров', 'pe@m.ru'),
 (3, 'Генадий', 'Фролов', 'fr@v.ru'),
 (2, 'рлорлорлор', 'Сидорова', 'si@m.ru')]
[(1, 1, '777777'), (2, 2, '222222'), (3, 3, '333333'), (4, 1, '666666')]
